In [2]:
import os
if not os.path.isdir('k'):
 !git clone -b development16 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
!cd k && pip install .

Cloning into 'k'...


^C


Processing c:\users\dddl distributed\documents\keffnet_learning\k
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pandas-2.0.1-cp39-cp39-win_amd64.whl (10.7 MB)
     ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
     --------------------------------------- 0.0/23.9 MB 495.5 kB/s eta 0:00:49
     --------------------------------------- 0.1/23.9 MB 469.7 kB/s eta 0:00:51
     --------------------------------------- 0.1/23.9 MB 525.1 kB/s eta 0:00:46
     --------------------------------------- 0.1/23.9 MB 469.7 kB/s eta 0:00:51
     --------------------------------------- 0.1/23.9 MB 481.4 kB/s eta 0:00:50
     --------------------------------------- 0.2/23.9 MB 482.7 kB/s eta 0:00:50
     --------------------------------------- 0.2/23.9 MB 456.4 kB/s eta 0:

In [6]:
import cai.layers
import cai.datasets
import cai.efficientnet
import cai.util
import gc
import multiprocessing
import random
import tensorflow as tf
import numpy as np
import matplotlib.pylab as plt
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.datasets
from tensorflow.keras import regularizers
import psutil

print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
print("CPU cores:", multiprocessing.cpu_count())
print('RAM:', (psutil.virtual_memory().total / 1e9),'GB')
print(tf.config.list_physical_devices('GPU'))

Tensorflow version: 2.10.1
Keras version: 2.10.0
CPU cores: 20
RAM: 137.24317696 GB
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [7]:
batch_size=32 # @param [16, 32, 64, 128, 256, 512] {type:"raw"} 
channels_per_group=16 # @param [2, 4, 8, 12, 16, 32] {type:"raw"}
epochs=50 # @param [2, 25, 50, 75, 100, 100, 200, 400] {type:"raw"}
verbose=True #@param {type:"boolean"}
bipolar_input=True #@param {type:"boolean"}
seed=7

global_input_shape = (32, 32, 1)

num_classes = 3

if channels_per_group==2:
   kTypes = [cai.layers.D6v3_2ch()]
elif channels_per_group==4:
   kTypes = [cai.layers.D6v3_4ch()]
elif channels_per_group==8:
   kTypes = [cai.layers.D6v3_8ch()]
elif channels_per_group==12:
   kTypes = [cai.layers.D6v3_12ch()]
elif channels_per_group==16:
   kTypes = [cai.layers.D6v3_16ch()]
else:
   kTypes = [cai.layers.D6v3_32ch()]

In [ ]:
from sklearn.metrics import balanced_accuracy_score

In [ ]:
def work_on_efficientnetV2(show_model=False, run_fit=False, test_results=False, test_part, calc_f1=False, kTypes=[]):
  monitor='val_loss'
  if (show_model):
    input_shape = global_input_shape
  else:
    input_shape = (None, None, global_input_shape[2])
  for kType in kTypes:
      basefilename = 'kEffNetV2-'+str(kType) + str(batch_size)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      model = cai.efficientnet.kEfficientNetB0(
        include_top=True,
        skip_stride_cnt=3,
        input_shape=input_shape,
        classes=num_classes,
        kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=[balanced_accuracy_score])

      if (show_model): 
        model.summary(line_length=180)

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit): 
            #train_flow = train_datagen.flow(
            #    X_train, y_train,
            #    batch_size=batch_size,
            #    shuffle=True,
            #    seed=seed
            #)
            #validation_flow = valid_datagen.flow(
            #    X_val, y_val,
            #    batch_size=batch_size,
            #    shuffle=False,
            #    seed=seed
            #)
            history = model.fit(
              X_train, y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data= (X_val, y_val),
              callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
              workers=cpus_num,
              max_queue_size=128
            )
            plt.figure()
            plt.ylabel("Accuracy (training and validation)")
            plt.xlabel("Epochs")
            plt.ylim([0,1])
            plt.plot(history.history["accuracy"])
            plt.plot(history.history["val_accuracy"])
      if (test_results):
        #test_flow = test_datagen.flow(
        #    X_test, y_test,
        #    batch_size=batch_size,
        #    shuffle=True,
        #    seed=seed
        #)
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            X_test, y_test,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      #if (calc_f1):
        #cai.datasets.test_flips_on_saved_model(X_test, y_test, best_result_file_name, has_flip_x=True, has_flip_y=True, has_bw=False, center_crop=0.15)
      print('Finished: '+basefilename)


In [ ]:
for i in range(10):
    basefilename = 'kEffNetV2-'+str(i+1)
    best_result_file_name = basefilename+'-best_result.hdf5'
    print('Running: '+basefilename)
    model = cai.efficientnet.kEfficientNetB0(
    include_top=True,
    skip_stride_cnt=3,
    input_shape=input_shape,
    classes=num_classes,
    kType=kType)
    
    optimizer = keras.optimizers.RMSprop()
    model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=[balanced_accuracy_score])